# LLM Evaluation Frameworks  
### TruthfulQA • LM Harness • HELM • OpenAI Evals  

**Author:** _Your Name_  
**Course:** Prompt Engineering – Evaluation Module  

---

This Colab notebook is a **hands‑on tour** of four major frameworks & datasets used to benchmark Large Language Models (LLMs).

| Framework / Dataset | Purpose | Key Metrics | Typical Use‑cases |
|--------------------|---------|-------------|-------------------|
| **TruthfulQA** | Dataset to test factual correctness vs. common misconceptions | Truthful accuracy, eval harness accuracy | Research on hallucinations & truthfulness |
| **LM Harness** | Unified wrapper around dozens of tasks / datasets | Task‑specific (accuracy, F1, BLEU, etc.) | Rapid leaderboard & model regression tests |
| **HELM** | Holistic evaluation across scenario, bias, robustness | Multiple: exact‑match, toxicity, robustness deltas | Transparency dashboards & broad model audits |
| **OpenAI Evals** | Flexible YAML + Python spec to score any model | User‑defined; integrates with OpenAI API | Custom evals, gated model & system tests |

By the end of this notebook you will be able to:

1. **Install** each tool in a fresh Colab runtime.  
2. **Run** a *minimal* evaluation against an open‑source or OpenAI model.  
3. **Interpret** the metrics produced.  
4. **Compare** trade‑offs when choosing an eval framework for class projects or production.

> ⚠️ **Note**: Full evaluations can take hours and cost money (OpenAI API).  We use _tiny subsets_ to keep runtimes & costs low for learning purposes.


## ⏬ Setup — Install Dependencies
Running the next cell installs lightweight versions of the required packages. If you’re on an Arm‑based Colab, add `--no-binary :all:` flags where needed.

In [ ]:
%%bash
pip -q install "lm-eval==0.4.0" "openai-evals==0.3.0" "helm-benchmark==0.4.1" --progress-bar off
python -m pip -q install --upgrade openai
echo '✅ Packages installed'

## 🔐 Configure Credentials
Only **OpenAI Evals** requires an API key. If you do not plan on using OpenAI models you can skip this step.

In [ ]:
import os, getpass, json, textwrap, warnings
os.environ['OPENAI_API_KEY'] = getpass.getpass('Paste your OpenAI API key (leave blank to skip): ')

## 1️⃣ TruthfulQA via LM Harness  

[TruthfulQA](https://github.com/sylinrl/TruthfulQA) challenges models on 817 questions designed to elicit **common false beliefs**.  
We run the **multiple‑choice** variant (`truthfulqa_mc`) on a small open‑source model for speed.

In [ ]:
from lm_eval import evaluator

results = evaluator.simple_evaluate(
    model="hf",
    model_args="pretrained=gpt2",  # tiny model for demo
    tasks=["truthfulqa_mc"],
    batch_size=4,
)

# Extract a key metric
truth_score = results["results"]["truthfulqa_mc"]["mc1"]
print(f"GPT‑2 TruthfulQA MC accuracy: {truth_score:.3f}")

### What to look for  
* **`mc1`**: accuracy when the highest probability *single* choice must be correct.  
Low scores on small models are expected. State‑of‑the‑art models (2025) reach >80 %.  
Try swapping `pretrained=gpt2` for `pretrained=meta-llama/Meta-Llama-3-8B` (requires 🤗 token).

## 2️⃣ LM Harness Quick‑Compare
The harness supports 200+ tasks. Below we sweep two small tasks to create a **mini leaderboard**.

In [ ]:
tasks = ["piqa", "hellaswag[:128]"]  # tiny slice for demo

results = evaluator.simple_evaluate(
    model="hf",
    model_args="pretrained=sshleifer/tiny-gpt2",
    tasks=tasks,
    batch_size=4,
)

from pprint import pprint
pprint({k: v for k, v in results["results"].items()})

Notice how **task adapters** abstract away dataset loading and metric computation. You can mix‑and‑match HuggingFace or OpenAI models via the same CLI.

## 3️⃣ HELM (Holistic Evaluation of Language Models)  

HELM provides **scenario‑based** benchmarking plus a rich HTML dashboard.  
Running the full suite is heavy (≈ 15 GB data). Instead we demonstrate a *single scenario* locally.

In [ ]:
%%bash
# Generate a stub run directory
helm-run   --suite accuracy --model gpt2 --scenario squad   --max-eval-instances 16   --output-dir helm_demo

# Display summary JSON
python - <<'PY'
import json, pathlib, pprint, os
summary = pathlib.Path("helm_demo/summary.json")
if summary.exists():
    data = json.loads(summary.read_text())
    pprint.pp({k:v for k,v in data.items() if k.startswith('metrics')})
else:
    print("HELM run may have failed. Reduce `--max-eval-instances` if RAM is low.")
PY

**HELM key ideas**

* **Scenario** = dataset + prompting strategy + metric set.  
* **Suite** = group of scenarios (e.g., *accuracy*, *robustness*, *bias*).  
* Generates an interactive **HTML report** (`index.html` in `helm_demo`). Download and open locally or via Colab `files.download`.

## 4️⃣ OpenAI Evals  

OpenAI Evals lets you define tests via **YAML + Jinja** (prompt templates) or pure Python.  
Below we clone a starter eval that checks a model’s ability to sort numbers.

In [ ]:
%%bash
evals init my_sort_eval --task simple_sort
cd my_sort_eval
echo "✅ Eval scaffold created"
ls -R

In [ ]:
%%bash
# Run the eval locally (uses your OPENAI_API_KEY)
cd my_sort_eval
evals run my_sort_eval   --model gpt-3.5-turbo-1106   --max_samples 10


The output prints **pass/fail counts** and logs.  
You can **register** your eval with `evals registry`.  
Try editing `prompts/prompt.jinja` to make the task harder, then re‑run.

## 🔄 Framework Comparison Cheat‑Sheet  

| Feature | LM Harness | TruthfulQA | HELM | OpenAI Evals |
|---------|------------|------------|------|--------------|
| **Install weight** | Light | Light (dataset only) | Heavy | Medium |
| **Task coverage** | 200+ | 1 (truthfulness) | 40+ scenarios | Custom by user |
| **Metrics** | Task‑specific | mc1, mc2, BLEU | Varied (accuracy → toxicity) | User‑defined |
| **Extensible** | ✅ YAML/JSON | Limited | ✅ Python | ✅ YAML + Python |
| **Dashboard** | CLI / JSON | CSV | HTML | JSON / OpenAI Console |
| **Best for** | Quick model sweeps | Hallucination studies | Holistic audits | CI/CD model gating |

**Take‑aways**

* Use **LM Harness** for *breadth* and leaderboard style comparison.  
* Add **TruthfulQA** when measuring *truthfulness/hallucination*.  
* Run **HELM** for research‑grade, multi‑axis audits (bias, robustness).  
* Embed **OpenAI Evals** into pipelines for regression testing and custom data.  


## 📝 Hands‑On Exercises  

1. **Swap models** – Replace `gpt2` with a newer model in Sections 1–2 and record the score delta.  
2. **Create a custom LM Harness task** – Follow the docs to wrap a JSONL dataset of your choice.  
3. **Add a bias scenario to HELM** – Adjust the `--suite` flag. Which metrics appear?  
4. **Write a new OpenAI Eval** – Build a math‑word‑problem checker and run it on `gpt‑4o`.  

Try at least one exercise before the next class and share screenshots of your results!

## 📚 Further Reading & Resources  

* TruthfulQA paper & dataset – Lin et al., 2022.  
* LM Evaluation Harness docs (GitHub).  
* HELM whitepaper – Stanford CRFM, 2022 → 2025 updates.  
* OpenAI Evals Cookbook example (`cookbook.openai.com`).  

---

© 2025 Prompt Engineering Course – Licensed CC‑BY‑SA  
